<a href="https://colab.research.google.com/github/deepak-ed/eq27ifuw/blob/main/Exercise_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Staying up-to-date

Hello everyone! Today we will work with *Tensorflow callbacks* to send status notifications using **Telegram**.


* **Telegram** is a free messaging app that focuses on speed 
and security, with over 500 million monthly active users.

In [1]:
import tensorflow as tf # Library for machine learning and AI 
import requests # Library used to send HTTP/1.1 requests

# Convolutional Neural Network

First, we need to create our cnn (see Exercise 4) to work later with our callbacks:

In [2]:
mnist = tf.keras.datasets.mnist #loading mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()  #assigning the test and train data
x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(8, (3,3), input_shape=(28,28,1), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(16, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [4]:
cnn.compile("adam", "sparse_categorical_crossentropy", metrics=['acc'])

# Telegram bot

## Create a callback that works using Telegram.

1. ToDo: Search how to create a Bot using Telegram and get its personal token.

In [5]:
personal_token = '5059385072:AAF-wE72L5x4qxp6kDFf7M8-zdhhefQ21Yw'

2. ToDo: Create the Tensorflow callback that notifies using the Telegram bot.

In [12]:
from types import new_class
# 2.1 ToDo: Declare the subclass to create a custom callback
class botCallback(tf.keras.callbacks.Callback):
  def __init__(self,personal_token):
    self.personal_token = personal_token
    self.ping_url = 'https://api.telegram.org/bot'+str(self.personal_token)+'/getUpdates'
    self.response = requests.get(self.ping_url).json()
    self.chat_id = self.response['result'][0]['message']['chat']['id']
    self.last_message_id = self.response['result'][-1]["message"]["message_id"]

  def send_message(self,message):
    self.ping_url = 'https://api.telegram.org/bot'+str(self.personal_token)+'/sendMessage?'+\
                    'chat_id='+str(self.chat_id)+\
                    '&parse_mode=Markdown'+\
                    '&text='+message
    self.response = requests.get(self.ping_url)


    #2.2 ToDo: Message notifying that it is starting to learn.
  def on_train_begin(self, logs = None):
    self.send_message("The training has started")
  
    #2.3 ToDo: Message notifying the results at the end of each epoch.
  def on_epoch_begin(self, epoch, logs = None):
    pass
      
  def on_epoch_end(self, epoch, logs = None):
    message = ' Epoch {}\n Training Accuracy : {:f}\n Training Loss : {:f}\n'.format(epoch,logs['acc'],logs['loss'])
    self.send_message(message)

      
    #2.4 ToDo: Message notifying that you are done training.
  def on_train_end(self, logs = None):
    self.send_message("The training has ended") 

    #2.5 ToDo: Message notifying the results of cnn.evaluate.
  def on_test_begin(self, logs=None):
    self.send_message("The testing has started")
  
  def on_test_end(self, logs=None):
    message = 'Testing Accuracy : {:f}\n Testing Loss : {:f}\n'.format(logs['acc'],logs['loss'])
    self.send_message(message)
    self.send_message("The testing has ended")
    #2.6 ToDo: Check the last message and if it is "stop" stop the training

  def on_epoch_begin(self, epoch, logs = None):
    pass
    
      
  def on_epoch_end(self, epoch, logs = None):
    message = ' Epoch {}\n Training Accuracy : {:f}\n Training Loss : {:f}\n'.format(epoch,logs['acc'],logs['loss'])
    self.send_message(message)
    self.ping_url = 'https://api.telegram.org/bot'+str(self.personal_token)+'/getUpdates'
    self.response = requests.get(self.ping_url).json()
    last_message = self.response['result'][-1]["message"]["text"]
    if(last_message== "stop"):
      self.model.stop_training = True
      message = 'last message is :{} '.format(last_message)
      self.send_message(message)
   


#Note: The Callback has to do every task asked, otherwise you don't get the point.

We assign the callback to a variable called ```bot_callback```:

In [13]:
bot_callback = botCallback(personal_token)

3. ToDo: make the callback work on both train and test:

In [10]:
h = cnn.fit(x_train[..., None], y_train, epochs=5, callbacks=[bot_callback])

Epoch 1/5
1875/1875 [==============================] - 35s 19ms/step - loss: 0.0316 - acc: 0.9901
Epoch 2/5
1875/1875 [==============================] - 36s 19ms/step - loss: 0.0262 - acc: 0.9914
Epoch 3/5
1875/1875 [==============================] - 36s 19ms/step - loss: 0.0204 - acc: 0.9932
Epoch 4/5
1875/1875 [==============================] - 35s 19ms/step - loss: 0.0171 - acc: 0.9944
Epoch 5/5
1875/1875 [==============================] - 35s 19ms/step - loss: 0.0154 - acc: 0.9948


In [14]:
cnn.evaluate(x_test, y_test, callbacks=[bot_callback])

313/313 [==============================] - 3s 11ms/step - loss: 0.0310 - acc: 0.9910


[0.03097439929842949, 0.9909999966621399]